In [1]:
# Import necessary libraries
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import LabelEncoder

# Sample user-item-rating data
data = {
    'user': ['Alice', 'Bob', 'Charlie', 'Alice', 'Bob', 'Alice', 'Charlie', 'Bob', 'Charlie', 'Alice'],
    'item': ['A', 'A', 'A', 'B', 'B', 'C', 'C', 'C', 'D', 'E'],
    'rating': [5, 4, 4, 3, 2, 4, 5, 1, 2, 5]
}

df = pd.DataFrame(data)

# Encode user and item names into numeric IDs for easier processing
user_encoder = LabelEncoder()
item_encoder = LabelEncoder()
df['user_id'] = user_encoder.fit_transform(df['user'])
df['item_id'] = item_encoder.fit_transform(df['item'])

# Create a user-item matrix (rows = users, columns = items, values = ratings)
user_item_matrix = df.pivot_table(index='user_id', columns='item_id', values='rating').fillna(0)

# Compute cosine similarity between users based on their ratings
user_similarity = cosine_similarity(user_item_matrix)

# Create a DataFrame for better readability of the similarity matrix
user_similarity_df = pd.DataFrame(user_similarity,
                                  index=user_encoder.classes_,
                                  columns=user_encoder.classes_)

print("User Similarity Matrix (Cosine Similarity):\n")
print(user_similarity_df.round(2))

# Select a target user for whom we want to make recommendations
target_user = 'Bob'
target_user_id = user_encoder.transform([target_user])[0]

# Get similarity scores of all users compared to the target user
similar_users = user_similarity[target_user_id]

# Get the indices of users sorted by similarity (excluding the target user)
similar_users_indices = similar_users.argsort()[::-1][1:]

top_similar_user_id = similar_users_indices[0]
top_ratings = user_item_matrix.iloc[top_similar_user_id]

already_rated = user_item_matrix.iloc[target_user_id] > 0

# Recommend items that the similar user rated but the target user hasn't rated
recommendations = top_ratings[~already_rated].sort_values(ascending=False)

# Convert item IDs back to original item labels and show top 3
recommended_items = item_encoder.inverse_transform(recommendations.index[:3])

print(f"\nTop Recommendations for {target_user}: {list(recommended_items)}")


User Similarity Matrix (Cosine Similarity):

         Alice   Bob  Charlie
Alice     1.00  0.76     0.69
Bob       0.76  1.00     0.68
Charlie   0.69  0.68     1.00

Top Recommendations for Bob: ['E', 'D']
